In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [2]:
#loading the data
train_data = np.load(r'datasets/Trousers & Jeans - All - Train.npz')
val_data = np.load(r'datasets/Trousers & Jeans - All - Validation.npz')
test_data = np.load(r'datasets/Trousers & Jeans - All - Test.npz')

In [3]:
train_images, train_labels = train_data['images'], train_data['labels']
val_images, val_labels = val_data['images'], val_data['labels']
test_images, test_labels = test_data['images'], test_data['labels']

In [4]:
#feature scaling
train_images = train_images/255.
val_images = val_images/255.
test_images = test_images/255.

In [5]:
#constant definition
NUM_EPOCHS = 18
BATCH_SIZE = 64

In [10]:
#hyperparam definition

HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(r'D:/sam/Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams= [HP_DROPOUT],
        metrics = [hp.Metric(METRIC_ACCURACY,display_name='accuracy')]
    )

In [6]:
#model definition and training
def train_test_model(hparams,session_num):
    
    #model outline
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,5, activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(64,3,activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(4)
    ])
    
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer='adam', loss= loss_fn, metrics=['accuracy'])
    
    #defining the logging dir
    log_dir = 'D:/sam/Logs/Model 1/fit/'+'run-{}'.format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label') 
        return figure

    def plot_to_image(figure):
    
        #save the plot to a png in memory
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
    
        #closing the figure to prevent from displaying in the notebook
        plt.close(figure)
    
        buf.seek(0)
    
        image = tf.image.decode_png(buf.getvalue(), channels=4)
    
        #add the batch dimension
        image = tf.expand_dims(image, 0)
    
        return image

    #file writer for logging cofusion matrix
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    def log_confusion_matrix(epoch, logs):
    #predict values for validation set
        test_pred_r = model.predict(val_images)
        test_pred = np.argmax(test_pred_r, axis=1)
    
        cm = sklearn.metrics.confusion_matrix(val_labels, test_pred)
    
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male', 'Jeans Male', 'Trousers Female', 'Jeans Female'])
        cm_image = plot_to_image(figure)
    
        #log the confusion matrix as image summary
        with file_writer_cm.as_default():
            tf.summary.image('Confusion Matrix', cm_image, step=epoch)
            
    
    
    #defining callbacks
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end= log_confusion_matrix)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode = 'auto',
        verbose=0,
        patience = 2,
        min_delta = 0,
        restore_best_weights= True
    )

    
    #training the model
    model.fit(
        train_images,
        train_labels,
        validation_data= (val_images,val_labels),
        verbose=2,
        epochs= NUM_EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks=[tensorboard_callback,cm_callback,early_stopping]
        
    )
    
    #model performance evaluation
    _, accuracy = model.evaluate(val_images,val_labels)
    
    #saving the model
    model.save(r'D:/sam/saved Models/Model 1/Run-{}'.format(session_num))
    
    return accuracy


In [7]:
#logging the result function
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [11]:
session_num = 1

for dropout in HP_DROPOUT.domain.values:
        
            hparams = {HP_DROPOUT : dropout}
        
            run_name = 'run-%d' %session_num
            print('---- Starting trial : %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('D:/sam/Logs/Model 1/hparam_tuning/' +run_name, hparams, session_num)
        
            session_num += 1

---- Starting trial : run-1
{'dropout': 0.0}
Epoch 1/18
63/63 - 51s - loss: 1.4007 - accuracy: 0.5065 - val_loss: 1.0255 - val_accuracy: 0.5680
Epoch 2/18
63/63 - 50s - loss: 0.8624 - accuracy: 0.6431 - val_loss: 0.8859 - val_accuracy: 0.6620
Epoch 3/18
63/63 - 50s - loss: 0.7651 - accuracy: 0.6851 - val_loss: 0.8530 - val_accuracy: 0.6520
Epoch 4/18
63/63 - 51s - loss: 0.7157 - accuracy: 0.7060 - val_loss: 0.8574 - val_accuracy: 0.6700
Epoch 5/18
63/63 - 52s - loss: 0.6773 - accuracy: 0.7263 - val_loss: 0.8448 - val_accuracy: 0.6500
Epoch 6/18
63/63 - 51s - loss: 0.6130 - accuracy: 0.7515 - val_loss: 0.8993 - val_accuracy: 0.6280
Epoch 7/18
63/63 - 52s - loss: 0.5963 - accuracy: 0.7617 - val_loss: 0.8012 - val_accuracy: 0.6860
Epoch 8/18
63/63 - 53s - loss: 0.5087 - accuracy: 0.7997 - val_loss: 0.7559 - val_accuracy: 0.7200
Epoch 9/18
63/63 - 51s - loss: 0.4683 - accuracy: 0.8109 - val_loss: 0.8321 - val_accuracy: 0.7180
Epoch 10/18
63/63 - 50s - loss: 0.4479 - accuracy: 0.8247 - val_

Epoch 14/18
63/63 - 49s - loss: 0.4246 - accuracy: 0.8329 - val_loss: 0.7208 - val_accuracy: 0.7620
16/16 [==============================] - 1s 85ms/step - loss: 0.7073 - accuracy: 0.7300
INFO:tensorflow:Assets written to: D:/sam/saved Models/Model 1/Run-7\assets
---- Starting trial : run-8
{'dropout': 0.7}
Epoch 1/18
63/63 - 53s - loss: 1.3095 - accuracy: 0.4903 - val_loss: 1.0529 - val_accuracy: 0.5400
Epoch 2/18
63/63 - 50s - loss: 0.9659 - accuracy: 0.6049 - val_loss: 0.9356 - val_accuracy: 0.6480
Epoch 3/18
63/63 - 52s - loss: 0.8893 - accuracy: 0.6409 - val_loss: 0.9861 - val_accuracy: 0.5920
Epoch 4/18
63/63 - 51s - loss: 0.8541 - accuracy: 0.6444 - val_loss: 0.9074 - val_accuracy: 0.6160
Epoch 5/18
63/63 - 52s - loss: 0.7856 - accuracy: 0.6846 - val_loss: 0.8743 - val_accuracy: 0.6220
Epoch 6/18
63/63 - 50s - loss: 0.7637 - accuracy: 0.6863 - val_loss: 0.8276 - val_accuracy: 0.6440
Epoch 7/18
63/63 - 50s - loss: 0.7145 - accuracy: 0.7100 - val_loss: 0.8349 - val_accuracy: 0.634

In [15]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"D:/sam/saved Models/Model 1/Run-9")

In [16]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)

16/16 [==============================] - 1s 82ms/step - loss: 0.6328 - accuracy: 0.7580


In [17]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.6328. Test accuracy: 75.80%


In [13]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 1/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [14]:
%load_ext tensorboard
%tensorboard --logdir "D:/sam/Logs/Model 1/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
